# Project 1 Starter

Project 1 is to allow students to practice Data Science concepts learned so far.

The project will include following tasks:
- Load dataset
- Clean up the data:
    - Encode replace missing values
    - Replace features values that appear incorrect
- Encode categorical variables
- Split dataset to Train/Test/Validation
- Add engineered features
- Train and tune ML model
- Provide final metrics using Validation dataset

It is up to you if you would like to modify your dataset and then split it, or split it then modify.
It is important to understand all the steps before model training, so that you can reliable replicate and test them to produce scoring function.

The Project-1 will be graded based on the completeness and performance of your final model against hold out dataset. 
The hold out dataset will not be known to the students. As part of your deliverables, you will be need to submit scoring function. The scoring function will perform following:
- Accept dataset in the same format as provided with the project, minus "MIS_Status" column
- Load trained model and any encoders that are needed to transform data
- Transform dataset into format that can be scored with the trained model
- Score the dataset and return the results, for each record
    - Record ID
    - Record label as determined by final model (0 or 1)
    - If your model returns probabilities, you need to assign label based on maximum F1 threshold


Deliverables:
- Jupyter notebook with complete code to manipulate data, train and tune final model
- Model and any potential encoders in the "pkl" format
- Scoring function that will load final model and encoders


Your notebook should include explanations about your code and be designed to be easily followed and results replicated. Once you are done with final version, you will need to test it by running all cells from top to bottom after restarting Kernel. It can be done by running `Kernel -> Restart & Run All`


**Important**: you might want to first produce working code using small subset of the dataset to speed up debuging process.

## Dataset description
The dataset for Lab-2 is sample of the SBA dataset posted on Kaggle.
The dataset is from the U.S. Small Business Administration (SBA) The U.S. SBA was founded in 1953 on the principle of promoting and assisting small enterprises in the U.S. credit market (SBA Overview and History, US Small Business Administration (2015)). Small businesses have been a primary source of job creation in the United States; therefore, fostering small business formation and growth has social benefits by creating job opportunities and reducing unemployment. There have been many success stories of start-ups receiving SBA loan guarantees such as FedEx and Apple Computer. However, there have also been stories of small businesses and/or start-ups that have defaulted on their SBA-guaranteed loans.  
More info on the original dataset: https://www.kaggle.com/mirbektoktogaraev/should-this-loan-be-approved-or-denied

**Don't use original dataset, use only dataset provided with project requirements in eLearning**

## Preparation

Use dataset provided in the eLearning

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 1500)

import warnings
warnings.filterwarnings('ignore')

#Extend cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
"""
Created on Mon Mar 18 18:25:50 2019

@author: Uri Smashnov

Purpose: Analyze input Pandas DataFrame and return stats per column
Details: The function calculates levels for categorical variables and allows to analyze summarized information

To view wide table set following Pandas options:
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth',200)
"""
import pandas as pd
def describe_more(df,normalize_ind=False, weight_column=None, skip_columns=[], dropna=True):
    var = [] ; l = [] ; t = []; unq =[]; min_l = []; max_l = [];
    assert isinstance(skip_columns, list), "Argument skip_columns should be list"
    if weight_column is not None:
        if weight_column not in list(df.columns):
            raise AssertionError('weight_column is not a valid column name in the input DataFrame')
      
    for x in df:
        if x in skip_columns:
            pass
        else:
            var.append( x )
            uniq_counts = len(pd.value_counts(df[x],dropna=dropna))
            uniq_counts = len(pd.value_counts(df[x], dropna=dropna)[pd.value_counts(df[x],dropna=dropna)>0])
            l.append(uniq_counts)
            t.append( df[ x ].dtypes )
            min_l.append(df[x].apply(str).str.len().min())
            max_l.append(df[x].apply(str).str.len().max())
            if weight_column is not None and x not in skip_columns:
                df2 = df.groupby(x).agg({weight_column: 'sum'}).sort_values(weight_column, ascending=False)
                df2['authtrans_vts_cnt']=((df2[weight_column])/df2[weight_column].sum()).round(2)
                unq.append(df2.head(n=100).to_dict()[weight_column])
            else:
                df_cat_d = df[x].value_counts(normalize=normalize_ind,dropna=dropna).round(decimals=2)
                df_cat_d = df_cat_d[df_cat_d>0]
                #unq.append(df[x].value_counts().iloc[0:100].to_dict())
                unq.append(df_cat_d.iloc[0:100].to_dict())
            
    levels = pd.DataFrame( { 'A_Variable' : var , 'Levels' : l , 'Datatype' : t ,
                             'Min Length' : min_l,
                             'Max Length': max_l,
                             'Level_Values' : unq} )
    #levels.sort_values( by = 'Levels' , inplace = True )
    return levels

### Load data

In [3]:
import numpy as np

In [4]:
data = pd.read_csv('SBA_loans_project_1.csv')           #store csv file in 'data' dataframe

In [5]:
print("Data shape:", data.shape)                       #Dimensions of the dataframe

Data shape: (809247, 20)


**Review dataset**

In [6]:
desc_df = describe_more(data)
desc_df 

,A_Variable,Levels,Datatype,Min Length,Max Length,Level_Values
0,City,31320,object,1,30,"{'LOS ANGELES': 10372, 'HOUSTON': 9260, 'NEW Y..."
1,State,51,object,2,3,"{'CA': 117341, 'TX': 63425, 'NY': 51877, 'FL':..."
2,Zip,32731,int64,1,5,"{10001: 841, 90015: 830, 93401: 729, 90010: 65..."
3,Bank,5716,object,3,30,"{'BANK OF AMERICA NATL ASSOC': 78111, 'WELLS F..."
4,BankState,55,object,2,3,"{'CA': 106293, 'NC': 71557, 'IL': 59258, 'OH':..."
5,NAICS,1307,int64,1,6,"{0: 181845, 722110: 25217, 722211: 17476, 8111..."
6,Term,407,int64,1,3,"{84: 207228, 60: 80965, 240: 77385, 120: 69852..."
7,NoEmp,581,int64,1,4,"{1: 138836, 2: 124470, 3: 81466, 4: 66306, 5: ..."
8,NewExist,3,float64,3,3,"{1.0: 580478, 2.0: 227709, 0.0: 932}"
9,CreateJob,234,int64,1,4,"{0: 566148, 1: 56789, 2: 52162, 3: 25945, 4: 1..."


## Dataset preparation and clean-up

Modify and clean-up the dataset as following:
- Replace encode Na/Null values
- Convert the strings styled as '$XXXX.XX' to float values. Columns = ['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']
- Convert MIS_Status to 0/1. Make value "CHGOFF" as 1

Replace encode Na/Null values

In [7]:
SBA=data.copy()                  #displaying the null values against each feature

In [8]:
list_item = []
for col in SBA.columns:
    list_item.append([col, SBA[col].dtype, SBA[col].isna().sum(), round((SBA[col].isna().sum()/len(SBA[col]))*100,2),
                      SBA[col].nunique(), list(SBA[col].sample(5).drop_duplicates().values)])

dfDesc = pd.DataFrame(columns=['feature', 'data_type', 'null', 'nulPct', 'unique', 'uniqueSample'],data=list_item)
dfDesc

,feature,data_type,null,nulPct,unique,uniqueSample
0,City,object,25,0.00,31320,"[Madison, ROGERS, CLAY CITY, PARKER, ENGLEWOOD]"
1,State,object,12,0.00,51,"[TX, VA, ID, IL, MS]"
2,Zip,int64,0,0.00,32731,"[3867, 66207, 60659, 34743, 8723]"
3,Bank,object,1405,0.17,5716,"[BBCN BANK, MOUNTAIN W. BANK A DIVISION OF, SE..."
4,BankState,object,1411,0.17,55,"[GA, IN, RI, OH, CA]"
5,NAICS,int64,0,0.00,1307,"[541990, 332812, 323119, 621999, 454390]"
6,Term,int64,0,0.00,407,"[240, 124, 73, 51, 40]"
7,NoEmp,int64,0,0.00,581,"[2, 24, 7, 1, 150]"
8,NewExist,float64,128,0.02,3,[1.0]
9,CreateJob,int64,0,0.00,234,"[0, 1, 8]"


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 809247 entries, 0 to 809246
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   City               809222 non-null  object 
 1   State              809235 non-null  object 
 2   Zip                809247 non-null  int64  
 3   Bank               807842 non-null  object 
 4   BankState          807836 non-null  object 
 5   NAICS              809247 non-null  int64  
 6   Term               809247 non-null  int64  
 7   NoEmp              809247 non-null  int64  
 8   NewExist           809119 non-null  float64
 9   CreateJob          809247 non-null  int64  
 10  RetainedJob        809247 non-null  int64  
 11  FranchiseCode      809247 non-null  int64  
 12  UrbanRural         809247 non-null  int64  
 13  RevLineCr          805153 non-null  object 
 14  LowDoc             806928 non-null  object 
 15  DisbursementGross  809247 non-null  object 
 16  Ba

### Filling the Null values with "Missing" and "0"

In [10]:
values_to_fill = {}
for col in data:
    if data[col].dtype == 'object':
        values_to_fill[col] = "Missing"
    else:
        values_to_fill[col] = 0

data.fillna(value=values_to_fill,inplace=True)
data.fillna(value=values_to_fill, inplace=True)

In [11]:
data.info()              #display showing no presence of null values in the data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 809247 entries, 0 to 809246
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   City               809247 non-null  object 
 1   State              809247 non-null  object 
 2   Zip                809247 non-null  int64  
 3   Bank               809247 non-null  object 
 4   BankState          809247 non-null  object 
 5   NAICS              809247 non-null  int64  
 6   Term               809247 non-null  int64  
 7   NoEmp              809247 non-null  int64  
 8   NewExist           809247 non-null  float64
 9   CreateJob          809247 non-null  int64  
 10  RetainedJob        809247 non-null  int64  
 11  FranchiseCode      809247 non-null  int64  
 12  UrbanRural         809247 non-null  int64  
 13  RevLineCr          809247 non-null  object 
 14  LowDoc             809247 non-null  object 
 15  DisbursementGross  809247 non-null  object 
 16  Ba

In [12]:
data.isna().sum()

City                 0
State                0
Zip                  0
Bank                 0
BankState            0
NAICS                0
Term                 0
NoEmp                0
NewExist             0
CreateJob            0
RetainedJob          0
FranchiseCode        0
UrbanRural           0
RevLineCr            0
LowDoc               0
DisbursementGross    0
BalanceGross         0
GrAppv               0
SBA_Appv             0
MIS_Status           0
dtype: int64

Convert the strings styled as '$XXXX.XX' to float values. Columns = ['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']

#### Defining a function to remove $ and , in the features that display currency and converting them to int data type

In [13]:
def currencyCleaning(x):
    x = x[1:].replace(',','')
    return x

In [14]:
data['DisbursementGross'] = data['DisbursementGross'].apply(currencyCleaning).astype(float).astype(int)
data['SBA_Appv'] = data['SBA_Appv'].apply(currencyCleaning).astype(float).astype(int)
data['BalanceGross'] = data['BalanceGross'].apply(currencyCleaning).astype(float).astype(int)
data['GrAppv'] = data['GrAppv'].apply(currencyCleaning).astype(float).astype(int)


In [15]:
data

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,GrAppv,SBA_Appv,MIS_Status
0,GLEN BURNIE,MD,21060,"BUSINESS FINANCE GROUP, INC.",VA,811111,240,7,1.0,6,7,1,1,0,N,743000,0,743000,743000,P I F
1,WEST BEND,WI,53095,JPMORGAN CHASE BANK NATL ASSOC,IL,722410,240,20,1.0,0,0,1,0,N,N,137000,0,137000,109737,P I F
2,SAN DIEGO,CA,92128,UMPQUA BANK,OR,0,120,2,1.0,0,0,1,0,0,N,280000,0,280000,210000,P I F
3,WEBSTER,MA,1570,HOMETOWN BANK A CO-OPERATIVE B,MA,621310,84,7,1.0,0,0,1,1,0,Y,144500,0,144500,122825,P I F
4,JOPLIN,MO,64804,U.S. BANK NATIONAL ASSOCIATION,OH,0,60,2,2.0,0,0,1,0,N,Y,52500,0,52500,42000,P I F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809242,BATON ROUGE,LA,70808,LOUISIANA CAP. CERT. DEVEL COM,LA,0,240,24,2.0,24,0,38605,0,N,N,749000,0,750000,750000,P I F
809243,GREENVILLE,SC,29611,SUNTRUST BANK,GA,0,72,5,1.0,0,0,1,0,N,Y,75000,0,75000,67500,P I F
809244,ST. PAUL,MN,55114,SUNRISE BANKS NATL ASSOC,MN,424990,120,31,1.0,0,31,1,1,0,N,216900,0,216900,162675,P I F
809245,TUSCALOOSA,AL,35401,BK OF TUSCALOOSA A DIVISION O,AL,0,120,4,2.0,0,0,1,1,N,N,75000,0,75000,60000,P I F


Convert MIS_Status to 0/1. Make value "CHGOFF" as 1

In [16]:
data['MIS_Status'] = [1 if status == 'CHGOFF' else 0 for status in data.MIS_Status.values] 
data

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,GrAppv,SBA_Appv,MIS_Status
0,GLEN BURNIE,MD,21060,"BUSINESS FINANCE GROUP, INC.",VA,811111,240,7,1.0,6,7,1,1,0,N,743000,0,743000,743000,0
1,WEST BEND,WI,53095,JPMORGAN CHASE BANK NATL ASSOC,IL,722410,240,20,1.0,0,0,1,0,N,N,137000,0,137000,109737,0
2,SAN DIEGO,CA,92128,UMPQUA BANK,OR,0,120,2,1.0,0,0,1,0,0,N,280000,0,280000,210000,0
3,WEBSTER,MA,1570,HOMETOWN BANK A CO-OPERATIVE B,MA,621310,84,7,1.0,0,0,1,1,0,Y,144500,0,144500,122825,0
4,JOPLIN,MO,64804,U.S. BANK NATIONAL ASSOCIATION,OH,0,60,2,2.0,0,0,1,0,N,Y,52500,0,52500,42000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809242,BATON ROUGE,LA,70808,LOUISIANA CAP. CERT. DEVEL COM,LA,0,240,24,2.0,24,0,38605,0,N,N,749000,0,750000,750000,0
809243,GREENVILLE,SC,29611,SUNTRUST BANK,GA,0,72,5,1.0,0,0,1,0,N,Y,75000,0,75000,67500,0
809244,ST. PAUL,MN,55114,SUNRISE BANKS NATL ASSOC,MN,424990,120,31,1.0,0,31,1,1,0,N,216900,0,216900,162675,0
809245,TUSCALOOSA,AL,35401,BK OF TUSCALOOSA A DIVISION O,AL,0,120,4,2.0,0,0,1,1,N,N,75000,0,75000,60000,0


In [17]:
desc_df1 = describe_more(data)
desc_df1

,A_Variable,Levels,Datatype,Min Length,Max Length,Level_Values
0,City,31321,object,1,30,"{'LOS ANGELES': 10372, 'HOUSTON': 9260, 'NEW Y..."
1,State,52,object,2,7,"{'CA': 117341, 'TX': 63425, 'NY': 51877, 'FL':..."
2,Zip,32731,int64,1,5,"{10001: 841, 90015: 830, 93401: 729, 90010: 65..."
3,Bank,5717,object,3,30,"{'BANK OF AMERICA NATL ASSOC': 78111, 'WELLS F..."
4,BankState,56,object,2,7,"{'CA': 106293, 'NC': 71557, 'IL': 59258, 'OH':..."
5,NAICS,1307,int64,1,6,"{0: 181845, 722110: 25217, 722211: 17476, 8111..."
6,Term,407,int64,1,3,"{84: 207228, 60: 80965, 240: 77385, 120: 69852..."
7,NoEmp,581,int64,1,4,"{1: 138836, 2: 124470, 3: 81466, 4: 66306, 5: ..."
8,NewExist,3,float64,3,3,"{1.0: 580478, 2.0: 227709, 0.0: 1060}"
9,CreateJob,234,int64,1,4,"{0: 566148, 1: 56789, 2: 52162, 3: 25945, 4: 1..."


In [18]:
desc_df1.iloc[[8,12,13,14,19]]["Level_Values"]

8                 {1.0: 580478, 2.0: 227709, 0.0: 1060}
12                     {1: 423681, 0: 290804, 2: 94762}
13    {'N': 378424, '0': 231967, 'Y': 181011, 'T': 1...
14    {'N': 704515, 'Y': 99339, 'Missing': 2319, '0'...
19                               {0: 667398, 1: 141849}
Name: Level_Values, dtype: object

In [19]:
desc_df1.iloc[5]["Level_Values"]               # NAICS Level values

{0: 181845,
 722110: 25217,
 722211: 17476,
 811111: 13125,
 621210: 12624,
 624410: 9053,
 812112: 8342,
 561730: 7980,
 621310: 7846,
 812320: 7125,
 541110: 6345,
 445310: 6164,
 621111: 6054,
 721110: 6032,
 447110: 5954,
 713940: 5923,
 811121: 5632,
 453220: 5452,
 451110: 5383,
 484110: 5275,
 453998: 5091,
 541330: 4907,
 238990: 4875,
 445110: 4871,
 541940: 4803,
 811192: 4376,
 722410: 4114,
 236118: 4017,
 235110: 3878,
 238220: 3855,
 541511: 3844,
 524210: 3719,
 812990: 3692,
 541512: 3646,
 236115: 3479,
 238210: 3440,
 722213: 3416,
 484121: 3409,
 531210: 3406,
 812310: 3400,
 453110: 3339,
 444130: 3317,
 445120: 3265,
 541611: 3113,
 561720: 3109,
 448310: 3031,
 448120: 2957,
 235310: 2945,
 441120: 2730,
 442110: 2718,
 112320: 2702,
 445299: 2677,
 446110: 2575,
 812199: 2530,
 441310: 2509,
 541430: 2487,
 541990: 2424,
 621320: 2424,
 452990: 2392,
 233210: 2381,
 541810: 2381,
 442210: 2343,
 443120: 2265,
 541211: 2221,
 541519: 2188,
 541613: 2133,
 541310: 

##### Upon observing NAICS level values we see that NAICS is a six digit numeric code. 
#### So, we remove all the data with incomplete NAICS code--where, NAICS is zero

In [20]:
data[data['NAICS']==0]

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,GrAppv,SBA_Appv,MIS_Status
2,SAN DIEGO,CA,92128,UMPQUA BANK,OR,0,120,2,1.0,0,0,1,0,0,N,280000,0,280000,210000,0
4,JOPLIN,MO,64804,U.S. BANK NATIONAL ASSOCIATION,OH,0,60,2,2.0,0,0,1,0,N,Y,52500,0,52500,42000,0
10,SIGNAL HILL,CA,90806,"CITIBANK, N.A.",CA,0,300,3,1.0,0,0,0,0,N,N,100000,0,100000,90000,0
14,RUSKIN,FL,33570,"PNC BANK, NATIONAL ASSOCIATION",OH,0,240,3,1.0,0,0,1,0,N,N,185100,0,185100,138825,0
18,LENEXA,KS,66215,CSRA LOCAL DEVEL CORP,GA,0,240,25,1.0,15,0,1,0,N,N,538000,0,538000,538000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809234,EUREKA,CA,95501,WESTAMERICA BANK,CA,0,300,5,2.0,0,0,1,0,N,N,530000,0,530000,371000,0
809237,BRUMLEY,MO,65017,CENTRAL OZARKS DEVELOPMENT INC,MO,0,240,3,1.0,0,3,1,0,N,N,91000,0,91000,91000,0
809242,BATON ROUGE,LA,70808,LOUISIANA CAP. CERT. DEVEL COM,LA,0,240,24,2.0,24,0,38605,0,N,N,749000,0,750000,750000,0
809243,GREENVILLE,SC,29611,SUNTRUST BANK,GA,0,72,5,1.0,0,0,1,0,N,Y,75000,0,75000,67500,0


In [21]:
data.NAICS
NAICSTrue = 0
NAICSFalse = 0
for i in data.NAICS.astype(str):
    if len(i) == 6:
        NAICSTrue += 1
    else:
        NAICSFalse += 1
print(NAICSTrue)
print(NAICSFalse)

627402
181845


In [22]:
data = data[data.NAICS != 0]            # Single digit NAICS code has been excluded in data

In [23]:
data.NewExist.value_counts()

1.0    452525
2.0    174093
0.0       784
Name: NewExist, dtype: int64

#### According to the SBA Loans data,  the only acceptable values for NewExist are 1.0 and 2.0 . 
#### Here, 0.0 is incomplete categorisation of record. These two have been excluded.

In [24]:
data = data[(data.NewExist == 1.0) | (data.NewExist == 2.0)]
data.NewExist.value_counts()

1.0    452525
2.0    174093
Name: NewExist, dtype: int64

In [25]:
desc_df1.iloc[13]["Level_Values"]

{'N': 378424,
 '0': 231967,
 'Y': 181011,
 'T': 13693,
 'Missing': 4094,
 '1': 22,
 'R': 12,
 '`': 9,
 '2': 6,
 'C': 2,
 '7': 1,
 '.': 1,
 '3': 1,
 'A': 1,
 'Q': 1,
 '-': 1,
 '5': 1}

In [26]:
data.RevLineCr.value_counts()

N          243072
0          192898
Y          175374
T           13123
Missing      2111
1              13
R              12
2               5
`               4
C               2
A               1
Q               1
-               1
7               1
Name: RevLineCr, dtype: int64

According to the SBA Loans data, the only acceptable values for RevLineCr (Revolving Line of Credit) are N (No) and Y (Yes) .
Here, all other categories have no meaning as to the categorisation of record. All such values have been excluded.

In [27]:
data = data[(data.RevLineCr == 'Y') | (data.RevLineCr == 'N')]
data.RevLineCr.value_counts()

N    243072
Y    175374
Name: RevLineCr, dtype: int64

In [28]:
data.LowDoc.value_counts()

N          391985
Y           22599
Missing      2270
C             506
S             499
0             445
A              90
R              51
1               1
Name: LowDoc, dtype: int64

According to the SBA Loans data, the only acceptable values for LowDoc are N (No) and Y (Yes) .
Here, all other categories have no meaning as to the categorisation of record. All such values have been excluded.

In [29]:
data = data[(data.LowDoc == 'Y') | (data.LowDoc == 'N')]
data.LowDoc.value_counts()

N    391985
Y     22599
Name: LowDoc, dtype: int64

The view of data after preprocessing and cleaning

In [30]:
data

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,GrAppv,SBA_Appv,MIS_Status
1,WEST BEND,WI,53095,JPMORGAN CHASE BANK NATL ASSOC,IL,722410,240,20,1.0,0,0,1,0,N,N,137000,0,137000,109737,0
5,NEWTOWN,OH,45244,HAMILTON CNTY DEVEL COMPANY IN,OH,234110,240,5,1.0,2,0,1,0,N,N,52000,0,52000,52000,0
6,MISSION VIEJO,CA,92691,BANK OF AMERICA CALIFORNIA N.A,CA,445310,60,3,1.0,0,0,1,0,Y,N,50000,0,50000,25000,0
7,OSWEGO,IL,60543,JPMORGAN CHASE BANK NATL ASSOC,IL,812990,57,1,1.0,2,1,0,1,Y,N,38619,0,25000,12500,1
8,DECATUR,GA,30033,WELLS FARGO BANK NATL ASSOC,SD,561421,71,10,1.0,1,11,1,1,Y,N,32714,0,20000,10000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809232,LOUISVILLE,KY,40241,"PNC BANK, NATIONAL ASSOCIATION",DE,812320,60,32,1.0,4,32,1,1,Y,N,35000,0,35000,17500,0
809233,REYNOLDSBURG,OH,43068,JPMORGAN CHASE BANK NATL ASSOC,IL,722110,84,6,1.0,0,6,0,1,N,N,72900,0,72900,36450,0
809238,DALLAS,TX,75216,BANK OF AMERICA NATL ASSOC,NC,561499,68,1,1.0,0,1,0,1,N,N,14408,0,8000,4000,1
809239,NAPERVILLE,IL,60565,SMALL BUS. GROWTH CORP,IL,624410,240,38,1.0,52,0,1,0,N,N,579000,0,579000,579000,0


## Categorical variables encoding

Encode categorical variables using either one of the techniques below. Don't use LabelEncoder.
- One-hot-encoder for variables with less than 10 valid values. Name your new columns "Original_name"_valid_value
- (If using sklearn) Target encoder from the following library: https://contrib.scikit-learn.org/category_encoders/index.html . Name your new column "Original_name"_trg
- (If using H2O) Use H2O target encoder


Example of use for target encoder:
```
import category_encoders as ce

encoder = ce.TargetEncoder(cols=[...])

encoder.fit(X, y)
X_cleaned = encoder.transform(X_dirty)
```

In [31]:
pip install category_encoders                   #Install category encoders

Note: you may need to restart the kernel to use updated packages.


'Zip' has been grouped under 'int' datatype. Whereas 'Zip' is a categorical column. So, the Zip feature has been converted to String type to denote text data.

In [32]:
data["Zip"] = data["Zip"].apply(str)      

Target Encoding of the features

In [33]:
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce
from copy import deepcopy

'''Save original columns that need to be droped or not used
   Vave One-ho and Label encoders for future use
'''
cols_to_drop = []
cat_encoders = {}
for col in data.drop(columns=['MIS_Status']).columns:
    if data[col].dtype == 'object':
        print("Target Encode scaling of ", col)
        enc = ce.TargetEncoder()
        enc.fit(data[col],data['MIS_Status'])
        data[col+"_trg"] = enc.transform(data[[col]])
        cat_encoders[col] = [deepcopy(enc),"trg"]
        

        
    cols_to_drop.append(col)

Target Encode scaling of  City
Target Encode scaling of  State
Target Encode scaling of  Zip
Target Encode scaling of  Bank
Target Encode scaling of  BankState
Target Encode scaling of  RevLineCr
Target Encode scaling of  LowDoc


In [34]:
data

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,GrAppv,SBA_Appv,MIS_Status,City_trg,State_trg,Zip_trg,Bank_trg,BankState_trg,RevLineCr_trg,LowDoc_trg
1,WEST BEND,WI,53095,JPMORGAN CHASE BANK NATL ASSOC,IL,722410,240,20,1.0,0,0,1,0,N,N,137000,0,137000,109737,0,0.120000,0.142680,0.180328,0.325763,0.278525,0.181492,0.223414
5,NEWTOWN,OH,45244,HAMILTON CNTY DEVEL COMPANY IN,OH,234110,240,5,1.0,2,0,1,0,N,N,52000,0,52000,52000,0,0.168675,0.193257,0.111111,0.000000,0.192848,0.181492,0.223414
6,MISSION VIEJO,CA,92691,BANK OF AMERICA CALIFORNIA N.A,CA,445310,60,3,1.0,0,0,1,0,Y,N,50000,0,50000,25000,0,0.234375,0.264017,0.257732,0.093117,0.285439,0.258492,0.223414
7,OSWEGO,IL,60543,JPMORGAN CHASE BANK NATL ASSOC,IL,812990,57,1,1.0,2,1,0,1,Y,N,38619,0,25000,12500,1,0.164557,0.269047,0.342105,0.325763,0.278525,0.258492,0.223414
8,DECATUR,GA,30033,WELLS FARGO BANK NATL ASSOC,SD,561421,71,10,1.0,1,11,1,1,Y,N,32714,0,20000,10000,1,0.253769,0.298170,0.309091,0.184712,0.235712,0.258492,0.223414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809232,LOUISVILLE,KY,40241,"PNC BANK, NATIONAL ASSOCIATION",DE,812320,60,32,1.0,4,32,1,1,Y,N,35000,0,35000,17500,0,0.209466,0.205250,0.196078,0.234759,0.288167,0.258492,0.223414
809233,REYNOLDSBURG,OH,43068,JPMORGAN CHASE BANK NATL ASSOC,IL,722110,84,6,1.0,0,6,0,1,N,N,72900,0,72900,36450,0,0.236842,0.193257,0.215686,0.325763,0.278525,0.181492,0.223414
809238,DALLAS,TX,75216,BANK OF AMERICA NATL ASSOC,NC,561499,68,1,1.0,0,1,0,1,N,N,14408,0,8000,4000,1,0.241379,0.206885,0.375000,0.280954,0.296050,0.181492,0.223414
809239,NAPERVILLE,IL,60565,SMALL BUS. GROWTH CORP,IL,624410,240,38,1.0,52,0,1,0,N,N,579000,0,579000,579000,0,0.240602,0.269047,0.326923,0.000000,0.278525,0.181492,0.223414


Scaling the numeric variables

In [35]:
from sklearn.preprocessing import MinMaxScaler
num_scalers = {}
'''Scale only original columns'''
for col in data.drop(columns=['MIS_Status']).columns[0:19]:
  if pd.api.types.is_numeric_dtype(data[col].dtype):
    print("MinMax scale of ", col)
    scaler = MinMaxScaler()
    scaler.fit(data[[col]])
    data[col+"_sc"] = scaler.transform(data[[col]])
    
    num_scalers[col] = [deepcopy(scaler),"MinMax"]
    cols_to_drop.append(col)

MinMax scale of  NAICS
MinMax scale of  Term
MinMax scale of  NoEmp
MinMax scale of  NewExist
MinMax scale of  CreateJob
MinMax scale of  RetainedJob
MinMax scale of  FranchiseCode
MinMax scale of  UrbanRural
MinMax scale of  DisbursementGross
MinMax scale of  BalanceGross
MinMax scale of  GrAppv
MinMax scale of  SBA_Appv


In [36]:
from sklearn.model_selection import train_test_split

X has the independent variables and Y the target feature -MIS_Status

In [37]:
X = data.drop(columns=['MIS_Status']+cols_to_drop)
Y = data.MIS_Status

In [38]:
X

,City_trg,State_trg,Zip_trg,Bank_trg,BankState_trg,RevLineCr_trg,LowDoc_trg,NAICS_sc,Term_sc,NoEmp_sc,NewExist_sc,CreateJob_sc,RetainedJob_sc,FranchiseCode_sc,UrbanRural_sc,DisbursementGross_sc,BalanceGross_sc,GrAppv_sc,SBA_Appv_sc
1,0.120000,0.142680,0.180328,0.325763,0.278525,0.181492,0.223414,0.748216,0.455408,0.0020,0.0,0.000000,0.000000,0.00001,0.0,0.011969,0.0,0.027361,0.024364
5,0.168675,0.193257,0.111111,0.000000,0.192848,0.181492,0.223414,0.150549,0.455408,0.0005,0.0,0.000356,0.000000,0.00001,0.0,0.004543,0.0,0.010360,0.011534
6,0.234375,0.264017,0.257732,0.093117,0.285439,0.258492,0.223414,0.409053,0.113852,0.0003,0.0,0.000000,0.000000,0.00001,0.0,0.004368,0.0,0.009960,0.005533
7,0.164557,0.269047,0.342105,0.325763,0.278525,0.258492,0.223414,0.859084,0.108159,0.0001,0.0,0.000356,0.000225,0.00000,0.5,0.003374,0.0,0.004960,0.002756
8,0.253769,0.298170,0.309091,0.184712,0.235712,0.258492,0.223414,0.551170,0.134725,0.0010,0.0,0.000178,0.002477,0.00001,0.5,0.002858,0.0,0.003960,0.002200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809232,0.209466,0.205250,0.196078,0.234759,0.288167,0.258492,0.223414,0.858264,0.113852,0.0032,0.0,0.000712,0.007206,0.00001,0.5,0.003058,0.0,0.006960,0.003867
809233,0.236842,0.193257,0.215686,0.325763,0.278525,0.181492,0.223414,0.747849,0.159393,0.0006,0.0,0.000000,0.001351,0.00000,0.5,0.006369,0.0,0.014541,0.008078
809238,0.241379,0.206885,0.375000,0.280954,0.296050,0.181492,0.223414,0.551265,0.129032,0.0001,0.0,0.000000,0.000225,0.00000,0.5,0.001259,0.0,0.001560,0.000867
809239,0.240602,0.269047,0.326923,0.000000,0.278525,0.181492,0.223414,0.628266,0.455408,0.0038,0.0,0.009251,0.000000,0.00001,0.0,0.050584,0.0,0.115765,0.128647


We are to divide the data into three partitions:
1. Training data
2. Validation data
3. Test data

But, sklearn divides data into only train and test.
Here, we divided our data into 60% training data and 40% is named remaining data.
we split the remaining data (40%)  into 50% validation and 50% test data--each forming 20% of the original data

In [39]:
X_train, X_rem, y_train, y_rem = train_test_split(X,Y,train_size=0.6,random_state=42)

In [40]:
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

Displaying the dimensions of data -TRAIN. VALIDATION. TEST

In [41]:
print(X_train.shape), print(y_train.shape)
print(X_valid.shape), print(y_valid.shape)
print(X_test.shape), print(y_test.shape)

(248750, 19)
(248750,)
(82917, 19)
(82917,)
(82917, 19)
(82917,)


(None, None)

# Model Training

Depending on the model of your choice, you might need to use appropriate scaler for numerical variables.

Train at least two types of models from the below list.
If you use sklearn libraries:
- Logistic regression
- SVM
- Decision Tree

If you use H2O libraries:
- GLM
- SVM
- Naïve Bayes Classifier

#### SVM

In [42]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,f1_score
y_true=list(y_valid)
for degree in [2,3,4,5,7]:
    svm = SVC(kernel='poly', degree=degree, max_iter=700, verbose=False)
    svm.fit(X_train, y_train)
    y_valid_pred = svm.predict(X_valid)
    y_tr_pred = svm.predict(X_train)
    print("Trained model for degree:", degree)
    print(confusion_matrix(y_true, y_valid_pred))
    print("F1 for Train:", f1_score(y_train, y_tr_pred, average='weighted'))
    print("F1 for valid:", f1_score(y_valid, y_valid_pred, average='weighted'))

Trained model for degree: 2
[[17442 47797]
 [ 4681 12997]]
F1 for Train: 0.3828431436319707
F1 for Test: 0.38479525186501495
Trained model for degree: 3
[[ 2225 63014]
 [  955 16723]]
F1 for Train: 0.12645465888698948
F1 for Test: 0.12437318552054569
Trained model for degree: 4
[[  856 64383]
 [  278 17400]]
F1 for Train: 0.09397889120162121
F1 for Test: 0.09489046525453271
Trained model for degree: 5
[[ 2040 63199]
 [  983 16695]]
F1 for Train: 0.1199061700247019
F1 for Test: 0.11998605675603767
Trained model for degree: 7
[[ 1942 63297]
 [ 1108 16570]]
F1 for Train: 0.11766533986351455
F1 for Test: 0.1171830062891873


#### Decision Tree

The unconstrained decision tree has good F1-scores for train and valid data. It may be due to overfitting.

In [43]:
from sklearn.tree import DecisionTreeClassifier


dtc_max_depth = DecisionTreeClassifier(random_state=0, 
                                       criterion="entropy")
dtc_max_depth.fit(X_train, y_train)
y_valid_pred = dtc_max_depth.predict(X_valid)
y_tr_pred = dtc_max_depth.predict(X_train)
print("F1 for Train:", f1_score(y_train, y_tr_pred, average='weighted'))
f1 = f1_score(y_valid, y_valid_pred, average='weighted')
print("F1 for valid:", f1)
print("Test set confusion matrix:")
print(confusion_matrix(y_true, y_valid_pred))

F1 for Train: 1.0
F1 for Test: 0.9324659912614648
Test set confusion matrix:
[[62464  2775]
 [ 2822 14856]]


Decision trees are better for categorical data and it deals colinearity better than SVM.

## Model Tuning

Choose one model from the above list. You should provide reasoning on why you have picked the model over others. Perform tuning for the selected model:
- Hyper-parameter tuning. Your hyper-parameter search space should have at least 50 combinations.
- To avoid overfitting and provide you with reasonable estimate of model performance on hold-out dataset, you will need to split your dataset as following:
    - Train, will be used to train model
    - Validation, will be used to validate model each round of training
    - Testing, will be used to provide final performance metrics, used only once on the final model
- Feature engineering. You should add at least two engineered features.  For example, add feature which is combination of two features.
- If your model returns probability, calculate probability threshold to maximize F1. 

Hyper parameter tuning on the decision tree

In [44]:
from sklearn.tree import DecisionTreeClassifier

best_f1 = 0
best_params = {}
best_model = None
for max_depth in [2,4,6,8,10,12,100]:
    for ccp_alpha in [0,0.0005, 0.001, 0.0015, 0.002, 0.0025]:
        for criterion in ["gini","entropy"]:
            dtc = DecisionTreeClassifier(random_state=0, 
                                     criterion=criterion,
                                     max_depth=max_depth,
                                     ccp_alpha=ccp_alpha)

            dtc.fit(X_train, y_train)
            y_valid_pred = dtc.predict(X_valid)
            y_tr_pred = dtc.predict(X_train)
            print("Trained Decision Tree with (max_depth, ccp_alpha, criterion):(",max_depth, ccp_alpha, criterion,")")
            #print("F1 for Train:", f1_score(Y_tr, y_tr_pred, average='weighted'))
            f1 = f1_score(y_valid, y_valid_pred, average='weighted')
            if f1>best_f1:
                best_f1 = f1
                best_params["max_depth"] = max_depth
                best_params["ccp_alpha"] = ccp_alpha 
                best_params["criterion"] = criterion
                
            print("F1 for Test:", f1)
            #print("Test set confusion matrix:")
            #print(confusion_matrix(y_true, y_pred))

'''Re-train model with best params'''
print("Best params:", best_params)
best_model = DecisionTreeClassifier(random_state=0, 
                                     criterion="entropy",
                                     max_depth=best_params["max_depth"],
                                     ccp_alpha=best_params["ccp_alpha"])
best_model.fit(X_train, y_train)
y_test_pred = best_model.predict(X_test)
y_tr_pred = best_model.predict(X_train)
print("Trained Decision Tree with (max_depth, ccp_alpha, criterion):(",best_params["max_depth"], best_params["ccp_alpha"],best_params["criterion"],")")
print("F1 for Train:", f1_score(y_train, y_tr_pred, average='weighted'))
f1 = f1_score(y_test, y_test_pred, average='weighted')
print("F1 for Valid:", f1)
print("Valid set confusion matrix:")
print(confusion_matrix(y_valid, y_valid_pred))

Trained Decision Tree with (max_depth, ccp_alpha, criterion):( 2 0 gini )
F1 for Test: 0.8614975296466127
Trained Decision Tree with (max_depth, ccp_alpha, criterion):( 2 0 entropy )
F1 for Test: 0.8647090577985146
Trained Decision Tree with (max_depth, ccp_alpha, criterion):( 2 0.0005 gini )
F1 for Test: 0.8614975296466127
Trained Decision Tree with (max_depth, ccp_alpha, criterion):( 2 0.0005 entropy )
F1 for Test: 0.8647090577985146
Trained Decision Tree with (max_depth, ccp_alpha, criterion):( 2 0.001 gini )
F1 for Test: 0.8614975296466127
Trained Decision Tree with (max_depth, ccp_alpha, criterion):( 2 0.001 entropy )
F1 for Test: 0.8647090577985146
Trained Decision Tree with (max_depth, ccp_alpha, criterion):( 2 0.0015 gini )
F1 for Test: 0.8614975296466127
Trained Decision Tree with (max_depth, ccp_alpha, criterion):( 2 0.0015 entropy )
F1 for Test: 0.8647090577985146
Trained Decision Tree with (max_depth, ccp_alpha, criterion):( 2 0.002 gini )
F1 for Test: 0.8614975296466127
Tr

## Save all artifacts

Save all artifacts needed for scoring function:
- Trained model
- Encoders

You should restart your Kernel now to properly test scoring function

Saving the hyperparameter trained model , encoders and scalers

In [45]:
import pickle
filename = 'decision_tree_model.sav'
pickle.dump(dtc, open(filename, 'wb'))
 

In [46]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
result

0.9147822521316498

In [47]:
filename1 = 'categorical_encoders.sav'
pickle.dump(cat_encoders, open(filename1, 'wb'))
 

In [48]:
loaded_encoders = pickle.load(open(filename1, 'rb'))
loaded_encoders

{'City': [TargetEncoder(cols=['City']), 'trg'],
 'State': [TargetEncoder(cols=['State']), 'trg'],
 'Zip': [TargetEncoder(cols=['Zip']), 'trg'],
 'Bank': [TargetEncoder(cols=['Bank']), 'trg'],
 'BankState': [TargetEncoder(cols=['BankState']), 'trg'],
 'RevLineCr': [TargetEncoder(cols=['RevLineCr']), 'trg'],
 'LowDoc': [TargetEncoder(cols=['LowDoc']), 'trg']}

In [49]:
filename2 = 'scalers.sav'
pickle.dump(num_scalers, open(filename2, 'wb'))

In [50]:
loaded_scalers = pickle.load(open(filename2, 'rb'))
loaded_scalers

{'NAICS': [MinMaxScaler(), 'MinMax'],
 'Term': [MinMaxScaler(), 'MinMax'],
 'NoEmp': [MinMaxScaler(), 'MinMax'],
 'NewExist': [MinMaxScaler(), 'MinMax'],
 'CreateJob': [MinMaxScaler(), 'MinMax'],
 'RetainedJob': [MinMaxScaler(), 'MinMax'],
 'FranchiseCode': [MinMaxScaler(), 'MinMax'],
 'UrbanRural': [MinMaxScaler(), 'MinMax'],
 'DisbursementGross': [MinMaxScaler(), 'MinMax'],
 'BalanceGross': [MinMaxScaler(), 'MinMax'],
 'GrAppv': [MinMaxScaler(), 'MinMax'],
 'SBA_Appv': [MinMaxScaler(), 'MinMax']}

## Model Scoring

Write function that will load artifacts from above, transform and score on a new dataset.
Your function should return Python list of labels. For example: [0,1,0,1,1,0,0]


In [51]:
def project_1_scoring(data):
    """
    Function to score input dataset.
    
    Input: dataset in Pandas DataFrame format
    Output: Python list of labels in the same order as input records
    
    Flow:
        - Load artifacts
        - Transform dataset
        - Score dataset
        - Return labels
    
    """
    l = data.shape[0]
    return l*[0]

# The scoring function can be found in the separate Juptyer notebook attached

## The scoring function returns a python list of target classification as "0" or "1"

### Example of Scoring function

Don't copy the code as is. It is provided as an example only. 
- Function `train_model` - you need to focus on model and encoder saving:
    ```
    pickle.dump(obj=clf, file=log_reg_file)
    pickle.dump(obj=cat_encoders, file=encoders_file)
    ```
- Function `project_1_scoring` - you should have similar function with name `project_1_scoring`. The function will:
    - Get Pandas dataframe as parameter
    - Will load model and all needed encoders
    - Will perform needed manipulations on the input Pandas DF - in the exact same format as input file for the project, minus MIS_Status feature
    - Return probabilities as numpy array or Python list
